# Project: Wrangling and Analyze Data

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [1]:
import pandas as pd
import numpy as np
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_1 = twitter_archive

In [ ]:
df_1.head()

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [ ]:
import requests
import os
folder_name = 'image_predictions'
if not os.path.exists(folder_name):
 os.makedirs(folder_name)
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open(os.path.join(folder_name, url.split('/')[-1]), mode = 'wb') as file:
 file.write(response.content)
os.listdir(folder_name)

In [ ]:
image_predictions = pd.read_csv('image_predictions//image-predictions.tsv', sep='\t')
df_2 = image_predictions
df_2.head()

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [ ]:
# NOTE TO REVIEWER: this student was unable to get approval from Twitter,
# as such is using the downloaded tweet provided by Udacity
import json
tweet_list = []
with open('tweet_json .txt', 'r') as file:
     for line in file:
        tweet_id = json.loads(line)['id']
        retweet_count = json.loads(line)['retweet_count']
        favorite_count = json.loads(line)['favorite_count']
        tweet_list.append({'tweet_id': tweet_id, 'retweet_count': retweet_count, 'favorite_count': favorite_count})
tweet_list = pd.DataFrame(tweet_list, columns=['tweet_id', 'retweet_count', 'favorite_count'])

In [ ]:
df_3 = tweet_list
df_3.head()

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



In [ ]:
df_1.info()

In [ ]:
df_1.describe()

In [ ]:
df_2.info()

In [ ]:
df_2.describe()

In [ ]:
df_3.info()

In [ ]:
df_3.describe()

### Quality issues
1.The retweeted status_id, retweeted status user_id and retweeted status timestamp columns will be removed since they are all retweeted and we do not need the retweeted values.
2.The timestamp column that has a datatype of object should be changed to a timestamp datatype

3.All the different dog names that are not correct should be removed

4.Change all the tweet_id from the tables to a datatype of string or object

5.Ratings without images should be removed due to the project rules

6.none values should be removed and replaced with null

7.some ratings are not correct

8.some dog names are represented as none in the df_1 table

9.The in_reply to status_id and in-reply to user-id should be removed

### Tidiness issues
1.According to the project rules the retweeted columns will not be needed after we get rid of the retweeted portions

2.In the df_1 table, the dog stages should be in one column because variables should be in columns and observation in rows

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
# Make copies of original pieces of data
df_1_clean = df_1.copy()
df_2_clean = df_2.copy()
df_3_clean = df_3.copy()

### Issue #1: Remove retweeted values

#### Define: In the df_1 tables..the colums that has retweeted portions ..such values will be removed which is the one that is not empty


#### Code

In [ ]:
# Remove retweeted values
df_1_clean = df_1_clean[df_1_clean.retweeted_status_id.isnull()]
df_1_clean = df_1_clean[df_1_clean.retweeted_status_user_id.isnull()]
df_1_clean = df_1_clean[df_1_clean.retweeted_status_timestamp.isnull()]

In [ ]:
# drop retweeted values
df_1_clean = df_1_clean.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'], axis=1)

#### Test

In [ ]:
df_1_clean.info()

### Issue #2: data types(timestamp)

#### Define
 The 'timestamp' column should be of datetime data type, but it is in string format.

#### Code

In [ ]:
# change the datatype of timestamp
df_1_clean.timestamp = pd.to_datetime(df_1_clean.timestamp)


In [ ]:
# check if it has changed
df_1_clean.info()

### Issue #3: Change datatype(tweet_id)

### #Define: change the datatype of all the tweet_id

##### code

In [ ]:
# change the datatype of tweet_id
df_1_clean.tweet_id = df_1_clean.tweet_id.astype(str)
df_2_clean.tweet_id = df_2_clean.tweet_id.astype(str)
df_3_clean.tweet_id = df_3_clean.tweet_id.astype(str)


In [ ]:
# Merge the df_1 and df_2 table together on tweet_id
image_id = df_2_clean[['tweet_id']]
df_1_clean = pd.merge(df_1_clean,image_id,on='tweet_id')

In [ ]:
# check if it has been merged
df_1_clean.info()

### Issue #4: Replace none values with nan

#### Define   replace the none values with nan in the 'name' column

#### code

In [ ]:
# Replace none values with nan
df_1_clean['name'].replace('None', np.nan, inplace=True)

In [ ]:
# check the value_count of the name column
df_1_clean.name.value_counts()

### Issue #5: Replace none values with nan

#### Define   replace the none values with nan in the 'four dog stages' column

In [ ]:
# Replace none values with nan
df_1_clean['doggo'].replace('None', np.nan, inplace=True)

In [ ]:
# check the value_count
df_1_clean.doggo.value_counts()

In [ ]:
# Replace none values with nan
df_1_clean['floofer'].replace('None', np.nan, inplace=True)

In [ ]:
# check the value_count
df_1_clean.floofer.value_counts()

In [ ]:
# Replace none values with nan
df_1_clean['pupper'].replace('None', np.nan, inplace=True)

In [ ]:
# check the value_count
df_1_clean.pupper.value_counts()

In [ ]:
# Replace none values with nan
df_1_clean['puppo'].replace('None', np.nan, inplace=True)

In [ ]:
# check the value_count
df_1_clean.puppo.value_counts()

### Issue#6: Ratings without images should be removed due to the project rules

#### define Ratings without images should be removed

In [ ]:
# Create a new column 'rating'
df_1_clean['rating'] = df_1_clean.rating_numerator / df_1_clean.rating_denominator
df_1_clean = df_1_clean.drop(['rating_numerator','rating_denominator'],axis=1)

In [ ]:
# Check the value_count of the column 'rating'
df_1_clean.rating.value_counts()

### Issue#7: some ratings are not correct

#### Define some ratings values should be removed

In [ ]:
# Remove ratings < 2
df_1_clean = df_1_clean[df_1_clean.rating<2]

In [ ]:
# sort rating values
df_1_clean.rating.sort_values()

### Issue #8: Drop duplicates values

#### Define Drop duplicate values in the tweet_id column

In [ ]:
# drop duplicates valuesin tweet_id column
df_3_clean = df_3_clean.drop_duplicates(subset='tweet_id')


In [ ]:
# Merge the df_1 copy and df_3 copy tables together
df_1_clean =pd.merge(df_1_clean,df_3_clean,on='tweet_id',how ='inner')

In [ ]:
# Use df.info to check
df_1_clean.info()

### Issue #9: Remove the in-reply-to-status id and user-id

In [ ]:
# Remove the in-reply-to-status id and user-id
df_1_clean = df_1_clean[df_1_clean.in_reply_to_status_id.isnull()]
df_1_clean = df_1_clean[df_1_clean.in_reply_to_user_id .isnull()]

In [ ]:
df_1_clean = df_1_clean.drop(['in_reply_to_status_id','in_reply_to_user_id'], axis=1)

In [ ]:
# check if it has been removed
df_1_clean.info()

# Tidiness issues

### Issue #1: Dog stages should be in one column

#### Define The four dog stages shpuld be merged into one column and each of them dropped

In [ ]:
# fill the empty spaces in the column with nan values
df_1_clean['doggo'].fillna('', inplace=True)

In [ ]:
# fill the empty spaces in the column with nan values
df_1_clean['floofer'].fillna('', inplace=True)

In [ ]:
# fill the empty spaces in the column with nan values
df_1_clean['pupper'].fillna('', inplace=True)

In [ ]:
# fill the empty spaces in the column with nan values
df_1_clean['puppo'].fillna('', inplace=True)

In [ ]:
# Merge the four columns into one column named stages
df_1_clean['stage'] = df_1_clean['doggo'].str.strip() + ' ' + df_1_clean['floofer'].str.strip() + ' ' + df_1_clean['pupper'].str.strip() + ' ' + df_1_clean['puppo'].str.strip()


In [ ]:
# Drop the dog stages columns
df_1_clean=df_1_clean.drop(['doggo','floofer','pupper','puppo'],axis=1)

In [ ]:
df_1_clean.stage=df_1_clean.stage.replace('',np.nan,)

In [ ]:
# Check for the value count of stage column
df_1_clean.stage.value_counts()

In [ ]:
# check for info
df_1_clean.info()

In [ ]:
df_1_clean.stage=df_1_clean.stage.replace('',np.nan,)

In [ ]:
# replace empty values with nan
df_1_clean['stage'] = df_1_clean['stage'].astype(str)
df_1_clean['stage'] = df_1_clean['stage'].str.strip()
df_1_clean['stage'].replace('', np.nan, inplace=True)

In [ ]:
# Check for the sum of nan values
df_1_clean['stage'].isna().sum()

In [ ]:
# check if the nan values has been dropped
df_1_clean.stage.value_counts()

In [ ]:
# remove other names
to_remove = ['doggo  pupper','doggo   puppo', 'doggo floofer', '']
df_1_clean.loc[df_1_clean['stage'].isin(to_remove), 'stage'] = 'Other'

In [ ]:
# Check if the names has been removed
df_1_clean.stage.value_counts()

In [ ]:
# Check the info
df_1_clean.info()

### Issue #2:remove the retweeted count and favourite count columns

#### Define  remove the retweeted count and favourite count columns

In [ ]:
# remove the retweeted count and favourite count columns
df_1_clean = df_1_clean[df_1_clean['retweet_count'].notnull() & df_1_clean['favorite_count'].notnull()]

In [ ]:
# Check for samples
df_1_clean.sample(12)

In [ ]:
df_1_clean.rating.mean()

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [ ]:
df_1_clean.to_csv('twitter_archive_master.csv', index=False)

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.Among the four dog stages, \"pupper\" has the highest frequency, but also has the lowest number of favorites, retweets, and the lowest rating meaning pupper stages of dogs are either not appealing to the public

2.Each tweet has a unique ID represented by the 'tweet_id' column.

3.Posts with higher ratings tend to have more favorites and retweets.

4.The 'timestamp' column shows when the tweet was posted

### Visualization

In [ ]:
df_1_clean['rating'] = df_1_clean['rating'].round(1)

In [ ]:
df_1_clean['rating'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
df_1_clean.groupby(['rating'])['retweet_count'].mean().plot(kind='bar',title='Mean Retweet Count by Rating', color='green')
plt.xlabel('Rating')
plt.ylabel('Retweet Count')
plt.show();

In [ ]:
value = np.sort(df_1_clean.rating.unique())
rating_analyze = pd.DataFrame(columns=['rating','number','retweet','favorite'])
rating_analyze['rating'] = value
for x in rating_analyze['rating']:
    my_df = df_1_clean[df_1_clean.rating==x]
    rating_analyze.loc[rating_analyze.rating==x,'number']=my_df['tweet_id'].count()
    rating_analyze.loc[rating_analyze.rating==x,'retweet']= my_df.retweet_count.mean()
    rating_analyze.loc[rating_analyze.rating==x,'favorite']= my_df.favorite_count.mean()
rating_analyze[['rating','number']].plot(x='rating',kind='bar', title='Rating Distribution')
rating_analyze[['rating','retweet','favorite']].plot(x='rating',title='Rating-Retweet & Favorite Analysis')
